In [1]:
!pip install torch_geometric
!pip install biopython
!pip install networkx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 18.8 MB/s eta 0:00:00


In [2]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.data import Data, DataLoader
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import requests
import networkx as nx
from scipy.spatial.distance import euclidean
from Bio.PDB import PDBParser, PDBList
from torch_geometric.nn import GCNConv, GraphNorm, global_mean_pool

from torch.utils.data import Dataset

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import pandas as pd
import numpy as np

Mounted at /content/drive


In [29]:
# df_fireprot = pd.read_csv('/content/drive/MyDrive/BMI_707_Project/707_Files_for_Colab/df_with_skipped_flag.csv')

In [76]:
df_fireprot = pd.read_csv("/content/drive/MyDrive/707/data/fireprotdb_results.csv")

<ipython-input-76-080b10c7c33b>:1: DtypeWarning: Columns (23,24,25,26,29) have mixed types. Specify dtype option on import or set low_memory=False.
  df_fireprot = pd.read_csv("/content/drive/MyDrive/707/data/fireprotdb_results.csv")


In [83]:
df_fireprot_w_ddG = df_fireprot[df_fireprot["ddG"].notna()]
df_fireprot_w_ddG.shape

(39177, 35)

In [85]:
df_fireprot_no_dupl = df_fireprot_w_ddG.drop_duplicates()
df_fireprot_no_dupl.shape

(12131, 35)

In [141]:
df_fireprot_w_pnames = df_fireprot_no_dupl[df_fireprot_no_dupl["protein_name"].notna()]
df_fireprot_w_uniprot_id = df_fireprot_w_pnames[df_fireprot_w_pnames["uniprot_id"].notna()]
df_fireprot_w_uniprot_id.shape

(12090, 35)

In [156]:
subset_cols = ["protein_name","chain","position", "wild_type", "mutation"]
df_groupby = df_fireprot_w_uniprot_id.groupby(subset_cols)
df_fire_prot_median_ddG = pd.DataFrame()
exp_ids = []
p_names = []
pbd_ids = []
chains = []
positions = []
wild_types = []
mutations = []
ddGs = []
sequences = []
is_in_cat_pock = []
is_ess = []
for name, group in tqdm(df_groupby):
  median_ddG_id = group["ddG"].median()
  group["dist_to_median"] = group["ddG"].apply(lambda x: abs(x-median_ddG_id))
  group_sorted = group.sort_values(by="dist_to_median")
  exp_ids.append(group_sorted.iloc[0]["experiment_id"])
  p_names.append(group_sorted.iloc[0]["protein_name"])
  pbd_ids.append(group_sorted.iloc[0]["pdb_id"])
  positions.append(group_sorted.iloc[0]["position"])
  wild_types.append(group_sorted.iloc[0]["wild_type"])
  mutations.append(group_sorted.iloc[0]["mutation"])
  ddGs.append(group_sorted.iloc[0]["ddG"])
  sequences.append(group_sorted.iloc[0]["sequence"])
  is_in_cat_pock.append(group_sorted.iloc[0]["is_in_catalytic_pocket"])
  is_ess.append(group_sorted.iloc[0]["is_essential"])
  chains.append(group_sorted.iloc[0]["chain"])
  # print(group_sorted)

df_fire_prot_median_ddG = pd.DataFrame({
    "experiment_id": exp_ids,
    "protein_name": p_names,
    "pdb_id": pbd_ids,
    "chain": chains,
    "position": positions,
    "wild_type": wild_types,
    "mutation": mutations,
    "ddG": ddGs,
    "sequence": sequences,
    "is_in_catalytic_pocket": is_in_cat_pock,
    "is_essential": is_ess
})

100%|██████████| 5086/5086 [00:08<00:00, 572.25it/s]


In [157]:
def mutate_sequence(row):
  sequence = list(row["sequence"])
  # position = min(row["position"]-1, len(sequence))
  sequence[row["position"]-1] = row["mutation"]
  return "".join(sequence)

In [160]:
df_fire_prot_median_ddG["mt_sequence"] = df_fire_prot_median_ddG.apply(mutate_sequence, axis=1)

In [164]:
df_fire_prot_median_ddG["pdb_id_list"] = df_fire_prot_median_ddG["pdb_id"].apply(lambda x: list(set(x.split("|"))))

In [167]:
df_fire_prot_median_ddG["pdb_id_len"] = df_fire_prot_median_ddG["pdb_id_list"].apply(lambda x: len(x))
df_fire_prot_median_ddG.sort_values(by="pdb_id_len")

,experiment_id,protein_name,pdb_id,chain,position,wild_type,mutation,ddG,sequence,is_in_catalytic_pocket,is_essential,mt_sequence,pdb_id_list,pdb_id_len
5081,PT008738,U1 small nuclear ribonucleoprotein A,1OIA,A,86,Y,F,2.60,MAVPETRPNHTIYINNLNEKIKKDELKKSLYAIFSQFGQILDILVS...,False,False,MAVPETRPNHTIYINNLNEKIKKDELKKSLYAIFSQFGQILDILVS...,[1OIA],1
5080,VB01735,U1 small nuclear ribonucleoprotein A,1OIA,A,78,Y,F,0.00,MAVPETRPNHTIYINNLNEKIKKDELKKSLYAIFSQFGQILDILVS...,False,False,MAVPETRPNHTIYINNLNEKIKKDELKKSLYAIFSQFGQILDILVS...,[1OIA],1
5079,VB04793,U1 small nuclear ribonucleoprotein A,1OIA,A,78,Y,D,4.30,MAVPETRPNHTIYINNLNEKIKKDELKKSLYAIFSQFGQILDILVS...,False,False,MAVPETRPNHTIYINNLNEKIKKDELKKSLYAIFSQFGQILDILVS...,[1OIA],1
5078,PT008735,U1 small nuclear ribonucleoprotein A,1OIA,A,31,Y,S,2.30,MAVPETRPNHTIYINNLNEKIKKDELKKSLYAIFSQFGQILDILVS...,False,False,MAVPETRPNHTIYINNLNEKIKKDELKKSLSAIFSQFGQILDILVS...,[1OIA],1
5077,VB01730,U1 small nuclear ribonucleoprotein A,1OIA,A,31,Y,F,0.00,MAVPETRPNHTIYINNLNEKIKKDELKKSLYAIFSQFGQILDILVS...,False,False,MAVPETRPNHTIYINNLNEKIKKDELKKSLFAIFSQFGQILDILVS...,[1OIA],1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
898,PT000404,Dihydrofolate reductase,1RX4|1DDR|1DDR|1DYJ|5DFR,A,31,F,V,1.50,MISLIAALAVDRVIGMENAMPWNLPADLAWFKRNTLNKPVIMGRHT...,True,True,MISLIAALAVDRVIGMENAMPWNLPADLAWVKRNTLNKPVIMGRHT...,"[5DFR, 1DDR, 1DYJ, 1RX4]",4
966,PT014402,Dihydrofolate reductase,1RX4|1DDR|1DDR|1DYJ|5DFR,A,122,D,A,1.60,MISLIAALAVDRVIGMENAMPWNLPADLAWFKRNTLNKPVIMGRHT...,False,False,MISLIAALAVDRVIGMENAMPWNLPADLAWFKRNTLNKPVIMGRHT...,"[5DFR, 1DDR, 1DYJ, 1RX4]",4
965,PT002910,Dihydrofolate reductase,1RX4|1DDR|1DDR|1DYJ|5DFR,A,121,G,Y,0.20,MISLIAALAVDRVIGMENAMPWNLPADLAWFKRNTLNKPVIMGRHT...,False,False,MISLIAALAVDRVIGMENAMPWNLPADLAWFKRNTLNKPVIMGRHT...,"[5DFR, 1DDR, 1DYJ, 1RX4]",4
964,VB05467,Dihydrofolate reductase,1RX4|1DDR|1DDR|1DYJ|5DFR,A,121,G,V,1.10,MISLIAALAVDRVIGMENAMPWNLPADLAWFKRNTLNKPVIMGRHT...,False,False,MISLIAALAVDRVIGMENAMPWNLPADLAWFKRNTLNKPVIMGRHT...,"[5DFR, 1DDR, 1DYJ, 1RX4]",4


In [174]:
df_temp = df_fire_prot_median_ddG[df_fire_prot_median_ddG["experiment_id"] == "PT008738"]
print(df_temp["sequence"].values[0][85])

Y


In [175]:
test = "MAVPETRPNHTIYINNLNEKIKKDELKKSLYAIFSQFGQILDILVSRSLKMRGQAFVIFKEVSSATNALRSMQGFPFYDKPMRIQYAKTDSDIIA"
test[85]

'Y'

In [ ]:
columns_to_keep = [
        'experiment_id', 'protein_name', 'uniprot_id', 'pdb_id', 'chain',
        'position', 'wild_type', 'mutation', 'ddG', 'sequence',
        'is_in_catalytic_pocket', 'is_essential'
]
# df_fireprot = df_fireprot[columns_to_keep].dropna().drop_duplicates(subset=['experiment_id'])

In [ ]:
# take shyam's code to generate the dataset
# make it run for 100 sequences => time it takes to run
# understand what is taking the longest: calling the API to get the pdbs or building the graphs everytime
# iterrows might be inefficient: perhaps better to group by wt sequence, then maybe parallel compute the calls to the API

In [ ]:
# Extract PDB file from uniprot id (wild-type)

def fetch_alphafold_pdb(uniprot_id, save_dir="wt_pdbs"):
    os.makedirs(save_dir, exist_ok=True)
    url = f"https://alphafold.ebi.ac.uk/files/AF-{uniprot_id}-F1-model_v4.pdb"
    save_path = os.path.join(save_dir, f"{uniprot_id}.pdb")
    if not os.path.exists(save_path):
        r = requests.get(url)
        if r.status_code == 200:
            with open(save_path, 'wb') as f:
                f.write(r.content)
        else:
            raise ValueError(f"Could not fetch AlphaFold PDB for {uniprot_id}")
    return save_path

# For the mutations becuase the unique PDBs used in the database correspond to the mutant protein

def fetch_pdb_on_demand(pdb_id, save_dir="pdbs"):
    pdbl = PDBList()
    pdb_id = pdb_id.lower()
    os.makedirs(save_dir, exist_ok=True)
    filepath = pdbl.retrieve_pdb_file(pdb_id, pdir=save_dir, file_format='pdb')
    actual_pdb_path = os.path.join(save_dir, f"{pdb_id}.pdb")
    if not os.path.exists(actual_pdb_path):
        os.rename(filepath, actual_pdb_path)
    return actual_pdb_path


# Use the PDB to extract coordinates of residues (structural info)

def extract_residue_coords(pdb_file, chain_id):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure('protein', pdb_file)
    model = structure[0]
    coords = {}
    for residue in model[chain_id]:
        if 'CA' in residue:
            res_id = residue.get_id()[1]
            coords[res_id] = residue['CA'].get_coord()
    return coords

# build graph structure graph where nodes = residues, edges = distance (angstroms)

def build_protein_graph(coords, distance_threshold=8.0):
    G = nx.Graph()
    residue_ids = list(coords.keys())
    for res_id in residue_ids:
        G.add_node(res_id, pos=coords[res_id])
    for i, id1 in enumerate(residue_ids):
        for id2 in residue_ids[i+1:]:
            dist = euclidean(coords[id1], coords[id2])
            if dist <= distance_threshold:
                G.add_edge(id1, id2, distance=dist)
    return G

    # 8.0 angstrom threshold back up by literature: https://pmc.ncbi.nlm.nih.gov/articles/PMC3549380/

# Construct the graph from predefined structure and populate with PDB extracted residue coordinates

def construct_graph_from_pdb(pdb_file, chain_id, distance_threshold=8.0):
    coords = extract_residue_coords(pdb_file, chain_id)
    return build_protein_graph(coords, distance_threshold)

# One hot encode the protein based on the 20 amino acids

def one_hot_encode_aa(aa):
    aa_list = list("ACDEFGHIKLMNPQRSTVWY")
    return [int(aa == x) for x in aa_list]

# Enrich the nodes with relevant functional features (amino acid type one-hot, catalytic pocket, position, relative pos, etc)

def enrich_graph_with_features(graph, row, sequence_length, mutate=False):
    sequence = row['sequence']
    catalytic = row['is_in_catalytic_pocket']
    essential = row['is_essential']
    mutation_pos = int(row['position'])
    mutant_aa = row['mutation']
    pdb_positions = sorted(graph.nodes)
    offset = min(pdb_positions)

    for pos in graph.nodes:
        seq_index = pos - offset
        if 0 <= seq_index < len(sequence):
            aa = sequence[seq_index]
            if mutate and pos == mutation_pos:
                aa = mutant_aa
            graph.nodes[pos]['aa_type'] = one_hot_encode_aa(aa)
            graph.nodes[pos]['is_catalytic'] = int(catalytic)
            graph.nodes[pos]['is_essential'] = int(essential)
            graph.nodes[pos]['relative_pos'] = pos / sequence_length
    return graph

# Convert the graph from nx to pytorch object for graph processing

def convert_nx_to_pyg(graph, ddG_value=0.0):
    features = []
    node_id_map = {}
    filtered_nodes = []
    for i, n in enumerate(graph.nodes):
        node_data = graph.nodes[n]
        if all(k in node_data for k in ['aa_type', 'is_catalytic', 'is_essential', 'relative_pos']):
            node_id_map[n] = len(filtered_nodes)
            filtered_nodes.append(n)
            feat = node_data['aa_type'] + [
                node_data['is_catalytic'],
                node_data['is_essential'],
                node_data['relative_pos']
            ]
            features.append(feat)
    if len(features) == 0:
        raise ValueError("No usable nodes with complete features.")
    edge_index, edge_attr = [], []
    for u, v, attrs in graph.edges(data=True):
        if u in node_id_map and v in node_id_map:
            edge_index.append([node_id_map[u], node_id_map[v]])
            edge_index.append([node_id_map[v], node_id_map[u]])
            edge_attr.append([attrs['distance']])
            edge_attr.append([attrs['distance']])
    if len(edge_index) == 0:
        raise ValueError("No valid edges in graph.")
    x = torch.tensor(features, dtype=torch.float)
    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
    edge_attr = torch.tensor(edge_attr, dtype=torch.float)
    y = torch.tensor([ddG_value], dtype=torch.float)
    return Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y)

In [ ]:
# Define DeltaDDGDataset class which extracts PDB structure and saves it per entry

# Note for future: consider storing the PDB files as a folder on Dropbox (.tar) for efficiency and speed. Store all PDB structures in one place.


# Global cache (can optionally persist to disk later)
structure_cache = {}

class DeltaDDGDataset(Dataset):
    def __init__(self, df):
        self.samples = []

        for _, row in tqdm(df.iterrows(), total=len(df), desc="Building dataset"):
            try:
                pdb_id = str(row['pdb_id']).split('|')[0].lower()
                uniprot_id = row['uniprot_id']

                # Cache structure fetches
                if pdb_id in structure_cache:
                    pdb_mut = structure_cache[pdb_id]
                else:
                    pdb_mut = fetch_pdb_on_demand(pdb_id)
                    structure_cache[pdb_id] = pdb_mut

                if uniprot_id in structure_cache:
                    pdb_wt = structure_cache[uniprot_id]
                else:
                    pdb_wt = fetch_alphafold_pdb(uniprot_id)
                    structure_cache[uniprot_id] = pdb_wt

                # Graph construction
                G_mut = construct_graph_from_pdb(pdb_mut, row['chain'])
                G_mut = enrich_graph_with_features(G_mut, row, len(row['sequence']), mutate=True)
                data_mut = convert_nx_to_pyg(G_mut, row['ddG'])
                data_mut.batch = torch.zeros(data_mut.num_nodes, dtype=torch.long)

                G_wt = construct_graph_from_pdb(pdb_wt, 'A')
                G_wt = enrich_graph_with_features(G_wt, row, len(row['sequence']), mutate=False)
                data_wt = convert_nx_to_pyg(G_wt, row['ddG'])
                data_wt.batch = torch.zeros(data_wt.num_nodes, dtype=torch.long)

                ddG = torch.tensor(row['ddG'], dtype=torch.float)
                metadata = row.to_dict()

                self.samples.append((data_wt, data_mut, ddG, metadata))

            except Exception as e:
                print(f"Skipping row due to error: {e}")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx]  # Always returns (data_wt, data_mut, ddG, metadata)

    def save(self, path):
        data = {
            'samples': self.samples,
        }
        torch.save(data, path)

    @classmethod
    def load(cls, path):
        data = torch.load(path, weights_only=False)
        obj = cls.__new__(cls)
        obj.samples = data['samples']
        return obj


In [ ]:
#Fusion Predictor Class for both ESM and GNN

from transformers import AutoTokenizer, EsmModel

class DDGFusionPredictor(nn.Module):
    def __init__(self,
                 esm_model_name="facebook/esm2_t6_8M_UR50D",
                 embedding_dim=320,
                 conv_channels=128,
                 heads=4,
                 ffn_dim=256,
                 hidden_dim = 256,
                 in_dim = 23):
        super().__init__()

        # Device setup for trainable layers
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Load pretrained ESM model on CPU
        self.tokenizer = AutoTokenizer.from_pretrained(esm_model_name)
        self.esm_model = EsmModel.from_pretrained(esm_model_name).eval()
        for param in self.esm_model.parameters():
            param.requires_grad = False

        self.conv1 = GCNConv(in_dim,    hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.conv3 = GCNConv(hidden_dim, hidden_dim)
        self.conv4 = GCNConv(hidden_dim, hidden_dim)

        self.gnorm1 = GraphNorm(hidden_dim)
        self.gnorm2 = GraphNorm(hidden_dim)
        self.gnorm3 = GraphNorm(hidden_dim)
        self.gnorm4 = GraphNorm(hidden_dim)
        self.embedding_dim = self.esm_model.config.hidden_size
        self.project = nn.Linear(hidden_dim, self.embedding_dim)

        # Convolutional encoder
        self.conv1d = nn.Conv1d(2*self.embedding_dim, conv_channels, kernel_size=3, padding=1)

        # Multi-head attention
        self.attention = nn.MultiheadAttention(embed_dim=conv_channels, num_heads=heads, batch_first=True)

        self.norm1 = nn.LayerNorm(conv_channels)
        self.norm2 = nn.LayerNorm(conv_channels)

        # Position-wise feedforward
        self.ffn = nn.Sequential(
            nn.Linear(conv_channels, ffn_dim),
            nn.ReLU(),
            nn.Linear(ffn_dim, conv_channels)
        )
        self.dropout = nn.Dropout(0.3)
        # Pooling layers
        self.global_avg_pool = nn.AdaptiveAvgPool1d(1)
        self.global_max_pool = nn.AdaptiveMaxPool1d(1)

        #Regression head
        self.regressor = nn.Sequential(
          nn.Linear(conv_channels * 2, 256),
          nn.ReLU(),
          nn.Dropout(0.3),
          nn.Linear(256, 64),
          nn.ReLU(),
          nn.Linear(64, 1)
       )


    def forward(self, wt_seq, mt_seq, data_wt, data_mut):
        # Tokenize and compute embeddings on CPU without gradients
        with torch.no_grad():
            wt_tokens = self.tokenizer(wt_seq, return_tensors="pt", padding=True, truncation=True,max_length=1024)
            mt_tokens = self.tokenizer(mt_seq, return_tensors="pt", padding=True, truncation=True,max_length=1024)

            wt_tokens = {k: v.to(self.device) for k, v in wt_tokens.items()}
            mt_tokens = {k: v.to(self.device) for k, v in mt_tokens.items()}

            wt_embed = self.esm_model(**wt_tokens).last_hidden_state[:, 1:-1, :]  # remove [CLS] and [SEP]
            mt_embed = self.esm_model(**mt_tokens).last_hidden_state[:, 1:-1, :]

            d = (wt_embed - mt_embed) # Move difference to GPU

        # Encde
        x0_wt = F.relu(self.gnorm1(self.conv1(data_wt.x, data_wt.edge_index)))
        x0_mut = F.relu(self.gnorm1(self.conv1(data_mut.x, data_mut.edge_index)))

        x1_wt = F.relu(self.gnorm2(self.conv2(x0_wt, data_wt.edge_index))) + x0_wt
        x1_mut = F.relu(self.gnorm2(self.conv2(x0_mut, data_mut.edge_index))) + x0_mut

        # x2 = F.relu(self.norm3(self.conv3(x1, data_wt.edge_index))) + x1
        x2_wt = F.relu(self.gnorm3(self.conv3(x1_wt, data_wt.edge_index))) + x1_wt
        x2_mut = F.relu(self.gnorm3(self.conv3(x1_mut, data_mut.edge_index))) + x1_mut

        x3_wt = F.relu(self.gnorm4(self.conv4(x2_wt, data_wt.edge_index))) + x2_wt
        x3_mut = F.relu(self.gnorm4(self.conv4(x2_mut, data_mut.edge_index))) + x2_mut

        h_wt = global_mean_pool(x3_wt, data_wt.batch)
        h_mut = global_mean_pool(x3_mut, data_mut.batch)

        end_g_encode_wt = self.project(h_wt)
        end_g_encode_mut = self.project(h_mut)

        d_g_encode = end_g_encode_wt - end_g_encode_mut


        B, L, D_esm = d.shape  # Original shape: [B, L, D_esm]

        # Reshape d_g_encode to [B, L, D_esm]
        d_g_encode_exp = d_g_encode.unsqueeze(1).expand(-1, L, D_esm)

        # conc_emb = torch.cat([d_t, d_g_encode_exp], dim=1)  # Original shape: [B, 2*D_esm, L]
        conc_emb = torch.cat([d, d_g_encode_exp], dim=2)  # Shape: [B, L, 2*D_esm]

        # Conv1D (expects B, C, L) and transpose back
        c = self.conv1d(conc_emb.transpose(1, 2)).transpose(1, 2)  # Expected shape: (B, L, C)

        # Multi-head attention + residual
        m, _ = self.attention(c, c, c)

        z = self.norm1(c + m)

        # Position-wise feedforward + residual
        f = self.norm2(z + self.dropout(self.ffn(z)))

        # Global pooling
        f_t = f.transpose(1, 2)  # (B, C, L)
        gp = self.global_avg_pool(f_t).squeeze(-1)  # (B, C)
        gm = self.global_max_pool(f_t).squeeze(-1)  # (B, C)
        conc = torch.cat([gp, gm], dim=1)  # (B, 2C)
        conc = self.dropout(conc)

        # Final regression output
        ddg_pred = self.regressor(conc).view(-1)
        return ddg_pred

In [ ]:
#load the data from saved files

train_dataset_loaded = DeltaDDGDataset.load("/content/drive/MyDrive//BMI_707_Project/707_Files_for_Colab/official_train_set.pkl")
val_dataset_loaded = DeltaDDGDataset.load("/content/drive/MyDrive//BMI_707_Project/707_Files_for_Colab/official_val_set.pkl")

#convert to dataloader object for training

# train_loader_official = DataLoader(train_dataset_loaded, batch_size=4, shuffle=True)
# val_loader_official = DataLoader(val_dataset_loaded, batch_size=4)


In [ ]:
#align 2 sequences
from Bio import Align
from Bio.Align import PairwiseAligner
import time

0.000522613525390625


In [ ]:
def get_alignment_from_graphs(graph, sequence):
  aa_feature_matrix = graph.x[:,:20]
  aa_np = aa_feature_matrix.numpy()
  aa_list_from_np = [idx2aa[idx] for idx in np.argmax(aa_np, axis=1)]
  aa_seq_from_np = "".join(aa_list_from_np)
  aligner = PairwiseAligner()
  aligner.mode = 'global'
  aligner.gap_score = -1
  aligner.target_gap_score = -float('inf')
  alignments = aligner.align(sequence,aa_seq_from_np)
  return alignments[0].indices[1,:]

# t = time.time()
for i in tqdm(range(len(train_dataset_loaded))):
  temp = train_dataset_loaded[i]
  wt_graph = temp[0]
  mut_graph = temp[1]
  metadata = temp[3]
  wt_seq = metadata["sequence"]
  mut_seq = list(wt_seq)
  mut_seq[metadata["position"]-1] = metadata["mutation"]
  mut_seq = "".join(mut_seq)
  wt_alignment = get_alignment_from_graphs(wt_graph, wt_seq)
  mut_alignment = get_alignment_from_graphs(mut_graph, mut_seq)
# print((time.time()-t)/100)

100%|██████████| 8221/8221 [00:28<00:00, 283.56it/s]


In [ ]:
train_dataset_loaded[-1]

(Data(x=[231, 23], edge_index=[2, 2004], edge_attr=[2004, 1], y=[1], batch=[231]),
 Data(x=[136, 23], edge_index=[2, 1334], edge_attr=[1334, 1], y=[1], batch=[136]),
 tensor(2.6000),
 {'experiment_id': 'VB05124',
  'protein_name': 'Thermonuclease',
  'uniprot_id': 'P00644',
  'pdb_id': '1STN|1EY0',
  'chain': 'A',
  'position': 183,
  'wild_type': 'E',
  'mutation': 'G',
  'ddG': 2.6,
  'sequence': 'MLVMTEYLLSAGICMAIVSILLIGMAISNVSKGQYAKRFFFFATSCLVLTLVVVSSLSSSANASQTDNGVNRSGSEDPTVYSATSTKKLHKEPATLIKAIDGDTVKLMYKGQPMTFRLLLVDTPETKHPKKGVEKYGPEASAFTKKMVENAKKIEVEFDKGQRTDKYGRGLAYIYADGKMVNEALVRQGLAKVAYVYKPNNTHEQHLRKSEAQAKKEKLNIWSEDNADSGQ',
  'is_in_catalytic_pocket': False,
  'is_essential': False})

In [ ]:
train_dataset_loaded[0]

(Data(x=[231, 23], edge_index=[2, 2004], edge_attr=[2004, 1], y=[1], batch=[231]),
 Data(x=[136, 23], edge_index=[2, 1334], edge_attr=[1334, 1], y=[1], batch=[136]),
 tensor(0.6000),
 {'experiment_id': 'VB05069',
  'protein_name': 'Thermonuclease',
  'uniprot_id': 'P00644',
  'pdb_id': '1STN|1EY0',
  'chain': 'A',
  'position': 164,
  'wild_type': 'T',
  'mutation': 'S',
  'ddG': 0.6,
  'sequence': 'MLVMTEYLLSAGICMAIVSILLIGMAISNVSKGQYAKRFFFFATSCLVLTLVVVSSLSSSANASQTDNGVNRSGSEDPTVYSATSTKKLHKEPATLIKAIDGDTVKLMYKGQPMTFRLLLVDTPETKHPKKGVEKYGPEASAFTKKMVENAKKIEVEFDKGQRTDKYGRGLAYIYADGKMVNEALVRQGLAKVAYVYKPNNTHEQHLRKSEAQAKKEKLNIWSEDNADSGQ',
  'is_in_catalytic_pocket': True,
  'is_essential': False})